### Notebook Overview: Illicit Subnetwork Analysis

This supplementary notebook analyses the structural characteristics of the illicit-only subnetworks generated in Model Step 3. It examines network-level metrics such as size, depth, connectivity, and transaction flow to understand the composition and variability of illicit Bitcoin activity across subnetworks.  

**Purpose**  
The purpose of this notebook is to quantify and summarise the key structural properties of the extracted subnetworks. By analysing attributes such as node count, edge count, and maximum path length (depth), it provides insights into how illicit activity propagates within the Bitcoin transaction graph. The analysis helps identify subnetworks that are more complex, interconnected, or influential—key indicators of organised laundering activity.  

**Key Steps**  
- Import illicit-only subnetworks and associated node rankings from BigQuery.  
- Compute network-level statistics for each subnetwork, including node count, edge count, and density.  
- Calculate graph-theoretic measures such as average degree, clustering coefficient, and connected component size.  
- Determine subnetwork depth and the longest transaction path using traversal-based analysis.  
- Summarise distributions of subnetwork sizes and depths to identify outliers or unusually large networks.  
- Export subnetwork metrics and summary tables to BigQuery for further reference or visualisation.  

**Outcome**  
The analysis shows that most illicit subnetworks are small and shallow, representing isolated or short transaction chains. A small subset of subnetworks exhibit greater depth and connectivity, indicating potential aggregation or layering behaviour consistent with money laundering typologies. The resulting subnetwork metrics enable prioritisation of complex or high-value subnetworks for ranking and visualisation in later investigative stages.  

**Context and Attribution**  
This notebook forms part of the technical work developed in support of the research thesis titled:  
_“Detection, Ranking and Visualisation of Money Laundering Networks on the Bitcoin Blockchain”_  
by Jennifer Payne (RMIT University).  

GitHub Repository: [https://github.com/majorpayne-2021/rmit_master_thesis](https://github.com/majorpayne-2021/rmit_master_thesis)  
Elliptic++ Dataset Source: [https://github.com/git-disl/EllipticPlusPlus](https://github.com/git-disl/EllipticPlusPlus)


In [1]:
# Data cleaning and manipulation
import pandas as pd
import numpy as np
import math
import time

# GCP libraries
from pandas_gbq import to_gbq # write pandas df to a GCP BigQuery table
import gcsfs
import importlib.util
import os
import inspect

# Set up display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Suppress FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


--------------
##### Read in Datasets
--------------


In [2]:
# Get new original (prior to duplication)
%%bigquery df_nw_txn_expanded
select * from `extreme-torch-467913-m6.networks.network_txn_expanded`;

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
df_nw_txn_expanded.head(1)

,txn_id,hop,subnetwork_id,seed_txn,merged_subnetwork_id
0,3084073,0,0,3084073,0


In [4]:
# Get nw summary
%%bigquery df_nw_summary
select * from `extreme-torch-467913-m6.networks.network_summary`;

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
df_nw_summary.head(1)

,merged_subnetwork_id,txn_ids,node_count,edge_count,depth,seeds,seed_count,linked_txn_count
0,9719,"[138779422, 274995025, 288474188, 306817114, 3...",186,201,9,"[306817114, 307096649, 308031465, 308871228, 3...",109,77


In [6]:
# Get txn nw list
%%bigquery df_nw_txn
select * from `extreme-torch-467913-m6.networks.network_txn_final`;

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
df_nw_txn.head(1)

,merged_subnetwork_id,txn_id,min_hop,seeds_in_group
0,121,51571237,5,[232359167]


--------------
##### Network Analysis
--------------


Total networks (original v deduplicated)

In [8]:
df_nw_txn_expanded['subnetwork_id'].nunique()

21020

In [9]:
df_nw_txn_expanded['merged_subnetwork_id'].nunique()

17864

In [10]:
df_nw_summary['merged_subnetwork_id'].nunique()

17864

Number of Nodes by Subnetwork

In [11]:
# 1) Pull the node counts as integers
s = pd.to_numeric(df_nw_summary["node_count"], errors="coerce").dropna().astype(int)
n = int(s.size)

# 2) Define bins: 1, 2, 3, 4, 5, and 5+
bins = [0, 1, 2, 3, 4, 5, float("inf")]   # 7 edges → 6 intervals
labels = ["1", "2", "3", "4", "5", "5+"]

# Group counts into bins
s_binned = pd.cut(s, bins=bins, labels=labels, right=True, include_lowest=True)

# 3) Distribution table
dist = (
    s_binned.value_counts()
    .sort_index()
    .rename_axis("Nodes in NW")
    .reset_index(name="Count of NW")
)
dist["% Population"] = (dist["Count of NW"] / n * 100).round(2)
dist["Cum Count"] = dist["Count of NW"].cumsum()
dist["Cum % Pop"] = (dist["Cum Count"] / n * 100).round(2)

# 4) Add totals row
totals = pd.DataFrame({
    "Nodes in NW": ["Total"],
    "Count of NW": [dist["Count of NW"].sum()],
    "% Population": [dist["% Population"].sum()],
    "Cum Count": [None],
    "Cum % Pop": [None]
})

dist = pd.concat([dist, totals], ignore_index=True)

dist


,Nodes in NW,Count of NW,% Population,Cum Count,Cum % Pop
0,1,14610,81.7800,14610,81.7800
1,2,2053,11.4900,16663,93.2800
2,3,488,2.7300,17151,96.0100
3,4,214,1.2000,17365,97.2100
4,5,120,0.6700,17485,97.8800
5,5+,379,2.1200,17864,100.0000
6,Total,17864,99.9900,None,NaN


Check if txn only belong to 1 subnetwork (mutually exclusive subnetworks).

In [12]:
# one row per (subnetwork, txn)
pairs = df_nw_txn[['merged_subnetwork_id','txn_id']].drop_duplicates()

# txn that appear in 2+ different subnetworks
conflicts = (pairs
             .groupby('txn_id')['merged_subnetwork_id']
             .nunique()
             .reset_index(name='num_subnets')
             .query('num_subnets > 1'))

has_conflicts = not conflicts.empty
print(f"Any conflicts? {has_conflicts}. # of conflicting txn: {len(conflicts)}")


Any conflicts? False. # of conflicting txn: 0


In [13]:
conflict_map = (pairs[pairs['txn_id'].isin(conflicts['txn_id'])]
                .groupby('txn_id')['merged_subnetwork_id']
                .agg(lambda x: sorted(set(x)))
                .rename('subnets')
                .reset_index())

print(conflict_map.head(20))


Empty DataFrame
Columns: [txn_id, subnets]
Index: []


Distance from seed node to destination node

In [14]:
import pandas as pd

# Assume df_nw_summary is already loaded

# 1) Pull depths as integers
s = pd.to_numeric(df_nw_summary["depth"], errors="coerce").dropna().astype(int)
n = int(s.size)

# 2) Define bins: 1, 2, 3, 4, and 5+
bins = [0, 1, 2, 3, 4, float("inf")]
labels = ["1", "2", "3", "4", "5+"]

# 3) Group into bins
s_binned = pd.cut(s, bins=bins, labels=labels, right=True, include_lowest=True)

# 4) Distribution table
depth_summary = (
    s_binned.value_counts()
    .sort_index()
    .rename_axis("depth_group")
    .reset_index(name="network_count")
)

# 5) Add percentages and cumulative stats
depth_summary["pct_population"] = (depth_summary["network_count"] / n * 100).round(2)
depth_summary["cum_count"] = depth_summary["network_count"].cumsum()
depth_summary["cum_pct_population"] = depth_summary["pct_population"].cumsum().round(2)

# 6) Add totals row
totals = pd.DataFrame({
    "depth_group": ["Total"],
    "network_count": [depth_summary["network_count"].sum()],
    "pct_population": [depth_summary["pct_population"].sum()],
    "cum_count": [depth_summary["cum_count"].iloc[-1]],
    "cum_pct_population": [depth_summary["cum_pct_population"].iloc[-1]]
})

depth_summary = pd.concat([depth_summary, totals], ignore_index=True)

depth_summary


,depth_group,network_count,pct_population,cum_count,cum_pct_population
0,1,16920,94.7200,16920,94.7200
1,2,517,2.8900,17437,97.6100
2,3,195,1.0900,17632,98.7000
3,4,103,0.5800,17735,99.2800
4,5+,129,0.7200,17864,100.0000
5,Total,17864,100.0000,17864,100.0000


Number of seeds per node

In [15]:
import pandas as pd

# Assume df_nw_summary is already loaded

# 1) Pull seed counts as integers
s = pd.to_numeric(df_nw_summary["seed_count"], errors="coerce").dropna().astype(int)
n = int(s.size)

# 2) Define bins: 1, 2, 3, 4, and 5+
bins = [0, 1, 2, 3, 4, float("inf")]
labels = ["1", "2", "3", "4", "5+"]

# 3) Group into bins
s_binned = pd.cut(s, bins=bins, labels=labels, right=True, include_lowest=True)

# 4) Distribution table
seed_summary = (
    s_binned.value_counts()
    .sort_index()
    .rename_axis("seed_group")
    .reset_index(name="network_count")
)

# 5) Add percentages and cumulative stats
seed_summary["pct_population"] = (seed_summary["network_count"] / n * 100).round(2)
seed_summary["cum_count"] = seed_summary["network_count"].cumsum()
seed_summary["cum_pct_population"] = seed_summary["pct_population"].cumsum().round(2)

# 6) Add totals row
totals = pd.DataFrame({
    "seed_group": ["Total"],
    "network_count": [seed_summary["network_count"].sum()],
    "pct_population": [seed_summary["pct_population"].sum()],
    "cum_count": [seed_summary["cum_count"].iloc[-1]],
    "cum_pct_population": [seed_summary["cum_pct_population"].iloc[-1]]
})

seed_summary = pd.concat([seed_summary, totals], ignore_index=True)

seed_summary


,seed_group,network_count,pct_population,cum_count,cum_pct_population
0,1,17378,97.2800,17378,97.2800
1,2,190,1.0600,17568,98.3400
2,3,77,0.4300,17645,98.7700
3,4,37,0.2100,17682,98.9800
4,5+,182,1.0200,17864,100.0000
5,Total,17864,100.0000,17864,100.0000


Matrix of count of nodes v depth of nodes for subnetworks with >= 2 nodes.

In [16]:
import pandas as pd

# --- 1) Filter to networks with >= 2 txns
df2 = df_nw_summary.loc[df_nw_summary["node_count"] >= 2,
                        ["merged_subnetwork_id", "node_count", "depth"]].copy()

# --- 2) Bin helpers
def bin_txn_count(x):
    x = int(x)
    if 1 <= x <= 5:
        return "1–5"
    elif 6 <= x <= 10:
        return "6–10"
    elif 11 <= x <= 15:
        return "11–15"
    elif 16 <= x <= 20:
        return "16–20"
    else:
        return "21+"

def bin_depth(x):
    x = int(x)
    if 1 <= x <= 5:
        return str(x)
    else:
        return "5+"

df2["txn_count_bin"] = df2["node_count"].map(bin_txn_count)
df2["depth_bin"] = df2["depth"].map(lambda d: bin_depth(max(1, int(d))))

# --- 3) Matrix: COUNT OF NETWORKS in each (txn_count_bin, depth_bin)
txn_categories = ["1–5", "6–10", "11–15", "16–20", "21+"]
depth_categories = ["1", "2", "3", "4", "5", "5+"]

matrix = pd.crosstab(
    index=df2["txn_count_bin"],
    columns=df2["depth_bin"]
).reindex(index=txn_categories, columns=depth_categories, fill_value=0)

matrix.index.name = "Txn Count (binned)"
matrix.columns.name = "Txn Depth (binned)"

# --- 4) Totals that will match the # of networks after filtering
total_networks = df2["merged_subnetwork_id"].nunique()
matrix_sum = int(matrix.values.sum())  # will equal total_networks since each network contributes exactly one cell

print(f"Unique subnetworks with ≥2 txns: {total_networks}")
print(f"Matrix cell sum (should match): {matrix_sum}")
matrix


Unique subnetworks with ≥2 txns: 3254
Matrix cell sum (should match): 3254


Txn Depth (binned),1,2,3,4,5,5+
Txn Count (binned),,,,,,
1–5,2277,429,121,48,0,0
6–10,19,56,33,32,30,34
11–15,7,18,14,11,4,15
16–20,3,3,11,3,3,8
21+,4,11,16,9,10,25
